In [1]:
import pandas as pd
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
from os import environ


In [2]:
from enum import Enum

class Set_date(Enum):
    master_start = "2017-10-01"
    master_end = "2024-05-31"
    #master_end = datetime.today().strftime('%Y-%m-%d')
    #master_end = "2020-02-29"
    
    period_start = "2023-10-01"
    #period_start = "2023-10-01"
    period_end = "2024-05-31"
    #period_end = datetime.today().strftime('%Y-%m-%d')
    #period_end = "2022-11-31"
    

In [3]:
from dotenv import load_dotenv
from decouple import config
load_dotenv("../aws.env")
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')


True

In [4]:
class InvalidHostException(Exception):
    """
        Raised when the host doesnt match the api version.
        Version 1 is mysql.haiti.org
        Version 2 is aws
    """
    pass


In [5]:
if environ['HOSTCaris'] != 'caris.cwyvkxmtzny2.us-east-1.rds.amazonaws.com':
    raise InvalidHostException ("Host is not compatible for the api/v2")

environ['HOSTCaris']


'caris.cwyvkxmtzny2.us-east-1.rds.amazonaws.com'

In [6]:
query_period = f"""
SELECT 
    a.id_patient,
    h.id_parenting_group,
    g.departement,
    g.commune,
    b.nbre_pres_for_inter,
    h.nbre_parenting_coupe_present,
    b.has_comdom_topic,
    b.has_preventive_vbg,
    d.number_of_condoms_sensibilize,
    d.number_condoms_sensibilization_date_in_the_interval,
    d.number_condoms_reception_in_the_interval,
    d.number_hiv_test_awareness_date_in_the_interval,
    d.type_of_test_vih,
    d.number_autotest_date_in_the_interval,
    d.number_test_date_in_the_interval,
    d.test_results,
    autotest_result,
    d.number_hiv_treatment_start_date_in_the_interval,
    d.number_vbg_treatment_date_in_the_interval,
    d.number_gynecological_care_date_in_the_interval,
    d.number_prep_awareness_date_in_the_interval,
    d.number_prep_reference_date_in_the_interval,
    d.number_prep_initiation_date_in_the_interval,
    d.number_contraceptive_sensibilization_date_in_the_interval,
    d.number_contraceptive_reception_in_the_interval,
    c.age_in_year,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 19,
            '15-19',
            IF(c.age_in_year >= 20
                    AND c.age_in_year <= 24,
                '20-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 17,
            '15-17',
            IF(c.age_in_year >= 18
                    AND c.age_in_year <= 24,
                '18-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    c.date_interview,
    IF(c.month_in_program >= 0
            AND c.month_in_program <= 6,
        '0-6 months',
        IF(c.month_in_program >= 7
                AND c.month_in_program <= 12,
            '07-12 months',
            IF(c.month_in_program >= 13
                    AND c.month_in_program <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    IF(sc.id_patient IS NOT NULL,
        'yes',
        'no') AS has_schooling_payment_in_the_interval,
    IF(e.id_patient IS NOT NULL,
        'yes',
        'no') AS muso,
    IF(f.id_patient IS NOT NULL,
        'yes',
        'no') AS gardening,
    IF(past.id_patient IS NOT NULL,
        'yes',
        'no') AS has_a_service_with_date_in_the_past
FROM
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.contraceptive_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.test_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.vbg_treatment_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.gynecological_care_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.prep_initiation_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.has_been_sensibilize_for_condom = 1
            AND ((dhi.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')))) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}') UNION (SELECT 
        dpga.id_patient
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}') UNION (SELECT 
        dm.id_patient
    FROM
        dream_member dm
    INNER JOIN patient p ON p.id = dm.id_patient
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm.id_patient) UNION (SELECT 
        dmx.id_patient
    FROM
        dream_member dmx
    INNER JOIN patient px ON px.id = dmx.id_patient
    INNER JOIN gardening_beneficiary gbx ON gbx.code_dreams = px.patient_code
    WHERE
    (gbx.date_modified BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
        AND (gbx.date_closed IS NULL)
    GROUP BY dmx.id_patient) UNION (SELECT 
        dmsch.id_patient
    FROM
        schooling_dreams sd
    LEFT JOIN dream_member dmsch ON dmsch.case_id = sd.parent_id
    WHERE
        ((sd.dat_peyman_fet BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            AND (sd.closed = 0))
    GROUP BY dmsch.id_patient)) a
        LEFT JOIN
    (SELECT 
        xy.id_patient,
            COUNT(*) AS nbre_pres_for_inter,
            IF((SUM(number_of_session_s_08) > 0
                OR SUM(number_of_session_s_10) > 0
                OR SUM(number_of_session_s_11) > 0
                OR SUM(number_of_session_s_18) > 0), 'yes', 'no') AS has_comdom_topic,
            IF((SUM(number_of_session_s_14) > 0
                OR SUM(number_of_session_s_16) > 0), 'yes', 'no') AS has_preventive_vbg
    FROM
        (SELECT 
        id_patient,
            SUM(dgs.topic = 8) AS number_of_session_s_08,
            SUM(dgs.topic = 10) AS number_of_session_s_10,
            SUM(dgs.topic = 11) AS number_of_session_s_11,
            SUM(dgs.topic = 18) AS number_of_session_s_18,
            SUM(dgs.topic = 14) AS number_of_session_s_14,
            SUM(dgs.topic = 16) AS number_of_session_s_16
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}'
    GROUP BY dga.id_patient , dgs.topic) xy
    GROUP BY xy.id_patient) b ON b.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient,
            TIMESTAMPDIFF(MONTH, dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817, NOW()) AS month_in_program,
            TIMESTAMPDIFF(YEAR, dsd.nan_ki_dat_ou_fet, NOW()) AS age_in_year,
            dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS date_interview
    FROM
        dream_member dm2
    LEFT JOIN dreams_surveys_data dsd ON dsd.case_id = dm2.case_id) c ON a.id_patient = c.id_patient
        LEFT JOIN
    (SELECT 
        dhi1.id_patient,
            SUM((dhi1.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.condom_sensibilization_date IS NOT NULL)) AS number_condoms_sensibilization_date_in_the_interval,
            SUM((dhi1.contraceptive_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.contraceptive_reception_date IS NOT NULL)) AS number_contraceptive_reception_in_the_interval,
            SUM(dhi1.has_been_sensibilize_for_condom = 1
                AND (dhi1.has_been_sensibilize_for_condom IS NOT NULL)) AS number_of_condoms_sensibilize,
            SUM((dhi1.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.condoms_reception_date IS NOT NULL)) AS number_condoms_reception_in_the_interval,
            SUM((dhi1.test_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.test_date IS NOT NULL)) AS number_test_date_in_the_interval,
            SUM((dhi1.vbg_treatment_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.vbg_treatment_date IS NOT NULL)) AS number_vbg_treatment_date_in_the_interval,
            SUM((dhi1.gynecological_care_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.gynecological_care_date IS NOT NULL)) AS number_gynecological_care_date_in_the_interval,
            SUM((dhi1.prep_initiation_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_initiation_date IS NOT NULL)) AS number_prep_initiation_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.test_result, ',') AS test_results,
            SUM((dhi1.prep_awareness_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_awareness_date IS NOT NULL)) AS number_prep_awareness_date_in_the_interval,
            SUM((dhi1.prep_reference_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_reference_date IS NOT NULL)) AS number_prep_reference_date_in_the_interval,
            SUM((dhi1.contraceptive_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.contraceptive_sensibilization_date IS NOT NULL)) AS number_contraceptive_sensibilization_date_in_the_interval,
            SUM((dhi1.hiv_treatment_start_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.hiv_treatment_start_date IS NOT NULL)) AS number_hiv_treatment_start_date_in_the_interval,
            SUM((dhi1.hiv_test_awareness_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.hiv_test_awareness_date IS NOT NULL)) AS number_hiv_test_awareness_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.type_of_test, ',') AS type_of_test_vih,
            SUM((dhi1.autotest_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.autotest_date IS NOT NULL)) AS number_autotest_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.autotest_result, ',') AS autotest_result
    FROM
        dream_hivinfos dhi1
    GROUP BY dhi1.id_patient) d ON a.id_patient = d.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient
    FROM
        dream_member dm2
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm2.id_patient
    GROUP BY dm2.id_patient) e ON a.id_patient = e.id_patient
        LEFT JOIN
    (SELECT 
    dm3.id_patient
FROM
    dream_member dm3
        INNER JOIN
    patient p1 ON p1.id = dm3.id_patient
        INNER JOIN
    gardening_beneficiary gb ON gb.code_dreams = p1.patient_code
WHERE
    (gb.date_modified BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
        AND (gb.date_closed IS NULL)
GROUP BY dm3.id_patient) f ON a.id_patient = f.id_patient
        LEFT JOIN
    (SELECT 
        dmy.id_patient, lc.name AS commune, ld.name AS departement
    FROM
        dream_member dmy
    LEFT JOIN dream_group dg ON dg.id = dmy.id_group
    LEFT JOIN dream_hub dh ON dh.id = dg.id_dream_hub
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement) g ON a.id_patient = g.id_patient
        LEFT JOIN
    (SELECT 
        dpga.id_patient,
            COUNT(*) AS nbre_parenting_coupe_present,
            dpgs.id_group AS id_parenting_group
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}'
    GROUP BY id_patient) h ON h.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dmsch.id_patient
    FROM
        schooling_dreams sd
    LEFT JOIN dream_member dmsch ON dmsch.case_id = sd.parent_id
    WHERE
        ((sd.dat_peyman_fet BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            AND (sd.closed = 0))
    GROUP BY dmsch.id_patient) sc ON sc.id_patient = a.id_patient
        LEFT JOIN
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.test_date < '{Set_date.period_start.value}')
            OR (dhi.condoms_reception_date < '{Set_date.period_start.value}')
            OR (dhi.vbg_treatment_date < '{Set_date.period_start.value}')
            OR (dhi.gynecological_care_date < '{Set_date.period_start.value}')
            OR (dhi.prep_initiation_date < '{Set_date.period_start.value}')
            OR (dhi.condom_sensibilization_date < '{Set_date.period_start.value}')
            OR (dhi.contraceptive_reception_date < '{Set_date.period_start.value}')) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date < '{Set_date.period_start.value}')) past ON past.id_patient = a.id_patient

"""

query_master = f"""
SELECT 
    a.id_patient,
    h.id_parenting_group,
    g.departement,
    g.commune,
    b.nbre_pres_for_inter,
    h.nbre_parenting_coupe_present,
    b.has_comdom_topic,
    b.has_preventive_vbg,
    d.number_of_condoms_sensibilize,
    d.number_condoms_sensibilization_date_in_the_interval,
    d.number_condoms_reception_in_the_interval,
    d.number_hiv_test_awareness_date_in_the_interval,
    d.type_of_test_vih,
    d.number_autotest_date_in_the_interval,
    d.number_test_date_in_the_interval,
    d.test_results,
    autotest_result,
    d.number_hiv_treatment_start_date_in_the_interval,
    d.number_vbg_treatment_date_in_the_interval,
    d.number_gynecological_care_date_in_the_interval,
    d.number_prep_awareness_date_in_the_interval,
    d.number_prep_reference_date_in_the_interval,
    d.number_prep_initiation_date_in_the_interval,
    d.number_contraceptive_sensibilization_date_in_the_interval,
    d.number_contraceptive_reception_in_the_interval,
    c.age_in_year,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 19,
            '15-19',
            IF(c.age_in_year >= 20
                    AND c.age_in_year <= 24,
                '20-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 17,
            '15-17',
            IF(c.age_in_year >= 18
                    AND c.age_in_year <= 24,
                '18-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    c.date_interview,
    IF(c.month_in_program >= 0
            AND c.month_in_program <= 6,
        '0-6 months',
        IF(c.month_in_program >= 7
                AND c.month_in_program <= 12,
            '07-12 months',
            IF(c.month_in_program >= 13
                    AND c.month_in_program <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    IF(sc.id_patient IS NOT NULL,
        'yes',
        'no') AS has_schooling_payment_in_the_interval,
    IF(e.id_patient IS NOT NULL,
        'yes',
        'no') AS muso,
    IF(f.id_patient IS NOT NULL,
        'yes',
        'no') AS gardening,
    IF(past.id_patient IS NOT NULL,
        'yes',
        'no') AS has_a_service_with_date_in_the_past
FROM
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.contraceptive_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.test_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.vbg_treatment_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.gynecological_care_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.prep_initiation_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.has_been_sensibilize_for_condom = 1
            AND ((dhi.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')))) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}') UNION (SELECT 
        dpga.id_patient
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}') UNION (SELECT 
        dm.id_patient
    FROM
        dream_member dm
    INNER JOIN patient p ON p.id = dm.id_patient
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm.id_patient) UNION (SELECT 
        dmx.id_patient
    FROM
        dream_member dmx
    INNER JOIN patient px ON px.id = dmx.id_patient
    INNER JOIN gardening_beneficiary gbx ON gbx.code_dreams = px.patient_code
    WHERE
    (gbx.date_modified BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
        AND (gbx.date_closed IS NULL)
    GROUP BY dmx.id_patient) UNION (SELECT 
        dmsch.id_patient
    FROM
        schooling_dreams sd
    LEFT JOIN dream_member dmsch ON dmsch.case_id = sd.parent_id
    WHERE
        ((sd.dat_peyman_fet BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            AND (sd.closed = 0))
    GROUP BY dmsch.id_patient)) a
        LEFT JOIN
    (SELECT 
        xy.id_patient,
            COUNT(*) AS nbre_pres_for_inter,
            IF((SUM(number_of_session_s_08) > 0
                OR SUM(number_of_session_s_10) > 0
                OR SUM(number_of_session_s_11) > 0
                OR SUM(number_of_session_s_18) > 0), 'yes', 'no') AS has_comdom_topic,
            IF((SUM(number_of_session_s_14) > 0
                OR SUM(number_of_session_s_16) > 0), 'yes', 'no') AS has_preventive_vbg
    FROM
        (SELECT 
        id_patient,
            SUM(dgs.topic = 8) AS number_of_session_s_08,
            SUM(dgs.topic = 10) AS number_of_session_s_10,
            SUM(dgs.topic = 11) AS number_of_session_s_11,
            SUM(dgs.topic = 18) AS number_of_session_s_18,
            SUM(dgs.topic = 14) AS number_of_session_s_14,
            SUM(dgs.topic = 16) AS number_of_session_s_16
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}'
    GROUP BY dga.id_patient , dgs.topic) xy
    GROUP BY xy.id_patient) b ON b.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient,
            TIMESTAMPDIFF(MONTH, dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817, NOW()) AS month_in_program,
            TIMESTAMPDIFF(YEAR, dsd.nan_ki_dat_ou_fet, NOW()) AS age_in_year,
            dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS date_interview
    FROM
        dream_member dm2
    LEFT JOIN dreams_surveys_data dsd ON dsd.case_id = dm2.case_id) c ON a.id_patient = c.id_patient
        LEFT JOIN
    (SELECT 
        dhi1.id_patient,
            SUM((dhi1.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.condom_sensibilization_date IS NOT NULL)) AS number_condoms_sensibilization_date_in_the_interval,
            SUM((dhi1.contraceptive_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.contraceptive_reception_date IS NOT NULL)) AS number_contraceptive_reception_in_the_interval,
            SUM(dhi1.has_been_sensibilize_for_condom = 1
                AND (dhi1.has_been_sensibilize_for_condom IS NOT NULL)) AS number_of_condoms_sensibilize,
            SUM((dhi1.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.condoms_reception_date IS NOT NULL)) AS number_condoms_reception_in_the_interval,
            SUM((dhi1.test_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.test_date IS NOT NULL)) AS number_test_date_in_the_interval,
            SUM((dhi1.vbg_treatment_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.vbg_treatment_date IS NOT NULL)) AS number_vbg_treatment_date_in_the_interval,
            SUM((dhi1.gynecological_care_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.gynecological_care_date IS NOT NULL)) AS number_gynecological_care_date_in_the_interval,
            SUM((dhi1.prep_initiation_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_initiation_date IS NOT NULL)) AS number_prep_initiation_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.test_result, ',') AS test_results,
            SUM((dhi1.prep_awareness_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_awareness_date IS NOT NULL)) AS number_prep_awareness_date_in_the_interval,
            SUM((dhi1.prep_reference_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_reference_date IS NOT NULL)) AS number_prep_reference_date_in_the_interval,
            SUM((dhi1.contraceptive_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.contraceptive_sensibilization_date IS NOT NULL)) AS number_contraceptive_sensibilization_date_in_the_interval,
            SUM((dhi1.hiv_treatment_start_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.hiv_treatment_start_date IS NOT NULL)) AS number_hiv_treatment_start_date_in_the_interval,
            SUM((dhi1.hiv_test_awareness_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.hiv_test_awareness_date IS NOT NULL)) AS number_hiv_test_awareness_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.type_of_test, ',') AS type_of_test_vih,
            SUM((dhi1.autotest_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.autotest_date IS NOT NULL)) AS number_autotest_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.autotest_result, ',') AS autotest_result
    FROM
        dream_hivinfos dhi1
    GROUP BY dhi1.id_patient) d ON a.id_patient = d.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient
    FROM
        dream_member dm2
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm2.id_patient
    GROUP BY dm2.id_patient) e ON a.id_patient = e.id_patient
        LEFT JOIN
    (SELECT 
    dm3.id_patient
FROM
    dream_member dm3
        INNER JOIN
    patient p1 ON p1.id = dm3.id_patient
        INNER JOIN
    gardening_beneficiary gb ON gb.code_dreams = p1.patient_code
WHERE
    (gb.date_modified BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
        AND (gb.date_closed IS NULL)
GROUP BY dm3.id_patient) f ON a.id_patient = f.id_patient
        LEFT JOIN
    (SELECT 
        dmy.id_patient, lc.name AS commune, ld.name AS departement
    FROM
        dream_member dmy
    LEFT JOIN dream_group dg ON dg.id = dmy.id_group
    LEFT JOIN dream_hub dh ON dh.id = dg.id_dream_hub
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement) g ON a.id_patient = g.id_patient
        LEFT JOIN
    (SELECT 
        dpga.id_patient,
            COUNT(*) AS nbre_parenting_coupe_present,
            dpgs.id_group AS id_parenting_group
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}'
    GROUP BY id_patient) h ON h.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dmsch.id_patient
    FROM
        schooling_dreams sd
    LEFT JOIN dream_member dmsch ON dmsch.case_id = sd.parent_id
    WHERE
        ((sd.dat_peyman_fet BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            AND (sd.closed = 0))
    GROUP BY dmsch.id_patient) sc ON sc.id_patient = a.id_patient
        LEFT JOIN
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.test_date < '{Set_date.master_start.value}')
            OR (dhi.condoms_reception_date < '{Set_date.master_start.value}')
            OR (dhi.vbg_treatment_date < '{Set_date.master_start.value}')
            OR (dhi.gynecological_care_date < '{Set_date.master_start.value}')
            OR (dhi.prep_initiation_date < '{Set_date.master_start.value}')
            OR (dhi.condom_sensibilization_date < '{Set_date.master_start.value}')
            OR (dhi.contraceptive_reception_date < '{Set_date.master_start.value}')) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date < '{Set_date.master_start.value}')) past ON past.id_patient = a.id_patient

"""

In [7]:
from sqlalchemy import create_engine, text
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")


In [8]:
agyw_served = pd.read_sql_query(text(query_master), engine.connect(), parse_dates=True)


In [9]:
agyw_served_period = pd.read_sql_query(text(query_period), engine.connect(), parse_dates=True)


In [10]:
# close the pool of connection
engine.dispose()


In [11]:
print(
    'period: ', 
    agyw_served_period.shape,
    'duplicated: ', 
    agyw_served_period.duplicated(subset=['id_patient']).sum()
)
print('master: ', 
agyw_served.shape,
'duplicated: ',
agyw_served.duplicated(subset=['id_patient']).sum()
)


period:  (6670, 34) duplicated:  0
master:  (60056, 34) duplicated:  0


In [12]:
actif_served = agyw_served[agyw_served.id_patient.isin(agyw_served_period.id_patient)]
actif_served.shape


(6670, 34)

In [13]:
actif_served.id_patient.isnull().unique()
actif_served[actif_served.id_patient.isna()]


array([False])

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,number_contraceptive_reception_in_the_interval,age_in_year,age_range,ovc_age,date_interview,month_in_program_range,has_schooling_payment_in_the_interval,muso,gardening,has_a_service_with_date_in_the_past


In [14]:
print(
    'period: ',
    actif_served.shape,
    'duplicated: ', 
    actif_served.duplicated(subset=['id_patient']).sum()
)


period:  (6670, 34) duplicated:  0


In [15]:
actif_served.insert(0,
'duplicated_id', 
actif_served.duplicated(subset=['id_patient'])
)


In [16]:
agyw_served.insert(0,
'duplicated_id', 
agyw_served.duplicated(subset=['id_patient'])
)


In [17]:
agyw_served_period.insert(0,
'duplicated_id',
agyw_served_period.duplicated(subset=['id_patient'])
)


In [18]:
actif_served.query('duplicated_id == True').shape


(0, 35)

In [19]:
agyw_served.query('duplicated_id == True').shape


(0, 35)

In [20]:
agyw_served_period.query('duplicated_id == True').shape


(0, 35)

> we have duplicaed values in the dataset. We need to remove them. the columns id_patient.

In [21]:
""" actif_served.to_excel('all_actifs_thisperiod.xlsx', index=False, na_rep='')
agyw_served.to_excel('all_actifs_sinceinception.xlsx', index=False, na_rep='')
agyw_served_period.to_excel('thisperiod_only.xlsx', index=False, na_rep='') """


" actif_served.to_excel('all_actifs_thisperiod.xlsx', index=False, na_rep='')\nagyw_served.to_excel('all_actifs_sinceinception.xlsx', index=False, na_rep='')\nagyw_served_period.to_excel('thisperiod_only.xlsx', index=False, na_rep='') "

# testing

In [22]:
from pandas import DataFrame

In [23]:
df = DataFrame({
    "groups":["A","B","C"],
    "data_1": [10,10,10],
    "data_2": [10,10,10]
})
df

,groups,data_1,data_2
0,A,10,10
1,B,10,10
2,C,10,10


In [24]:
df['Sub-Total'] = df[df.columns[-2:]].sum(axis=1)

In [25]:
df

,groups,data_1,data_2,Sub-Total
0,A,10,10,20
1,B,10,10,20
2,C,10,10,20


In [26]:
df[:-1]

,groups,data_1,data_2,Sub-Total
0,A,10,10,20
1,B,10,10,20
